**Imports**

In [ ]:
import gc
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from wordcloud import WordCloud
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import log_loss, r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from skopt import gp_minimize
from skopt.plots import plot_convergence
import json

%matplotlib inline

**Setup Environment**

In [ ]:
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()

**Get Training Data**

In [ ]:
(market_train, news_train) = env.get_training_data()

In [ ]:
market_train.describe()

In [ ]:
market_train.shape

**Exploratory Data Analysis**

**Market Data**
* time(datetime64[ns, UTC]) - the current time (in marketdata, all rows are taken at 22:00 UTC)
* assetCode(object) - a unique id of an asset
* assetName(category) - the name that corresponds to a group of assetCodes. These may be "Unknown" if the corresponding assetCode does not have any rows in the news data.
* universe(float64) - a boolean indicating whether or not the instrument on that day will be included in scoring. This value is not provided outside of the training data time period. The trading universe on a given date is the set of instruments that are avilable for trading (the scoring function will not consider instruments that are not in the trading universe). The trading universe changes daily.
* volume(float64) - trading volume in shares for the day
* close(float64) - the close price for the day (not adjusted for splits or dividends)
* open(float64) - the open price for the day (not adjusted for splits or dividends)
* returnsClosePrevRaw1(float64) - see returns explanation above
* returnsOpenPrevRaw1(float64) - see returns explanation above
* returnsClosePrevMktres1(float64) - see returns explanation above
* returnsOpenPrevMktres1(float64) - see returns explanation above
* returnsClosePrevRaw10(float64) - see returns explanation above
* returnsOpenPrevRaw10(float64) - see returns explanation above
* returnsClosePrevMktres10(float64) - see returns explanation above
* returnsOpenPrevMktres10(float64) - see returns explanation above
* returnsOpenNextMktres10(float64) - 10 day, market-residualized return. This is the target variable used in competition scoring. The market data has been filtered such that returnsOpenNextMktres10 is always not null.

**News Data**

* time(datetime64[ns, UTC]) - UTC timestamp showing when the data was available on the feed (second precision)
* sourceTimestamp(datetime64[ns, UTC]) - UTC timestamp of this news item when it was created
* firstCreated(datetime64[ns, UTC]) - UTC timestamp for the first version of the item
* sourceId(object) - an Id for each news item
* headline(object) - the item's headline
* urgency(int8) - differentiates story types (1: alert, 3: article)
* takeSequence(int16) - the take sequence number of the news item, starting at 1. For a given story, alerts and articles have separate sequences.
* provider(category) - identifier for the organization which provided the news item (e.g. RTRS for Reuters News, BSW for Business Wire)
* subjects(category) - topic codes and company identifiers that relate to this news item. Topic codes describe the news item's subject matter. These can cover asset classes, geographies, events, industries/sectors, and other types.
* audiences(category) - identifies which desktop news product(s) the news item belongs to. They are typically tailored to specific audiences. (e.g. "M" for Money International News Service and "FB" for French General News Service)
* bodySize(int32) - the size of the current version of the story body in characters
* companyCount(int8) - the number of companies explicitly listed in the news item in the subjects field
* headlineTag(object) - the Thomson Reuters headline tag for the news item
* marketCommentary(bool) - boolean indicator that the item is discussing general market conditions, such as "After the Bell" summaries
* sentenceCount(int16) - the total number of sentences in the news item. Can be used in conjunction with firstMentionSentence to determine the relative position of the first mention in the item.
* wordCount(int32) - the total number of lexical tokens (words and punctuation) in the news item
* assetCodes(category) - list of assets mentioned in the item
* assetName(category) - name of the asset
* firstMentionSentence(int16) - the first sentence, starting with the headline, in which the scored asset is mentioned.
1: headline
2: first sentence of the story body
3: second sentence of the body, etc
0: the asset being scored was not found in the news item's headline or body text. As a result, the entire news item's text (headline + body) will be used to determine the sentiment score.
* relevance(float32) - a decimal number indicating the relevance of the news item to the asset. It ranges from 0 to 1. If the asset is mentioned in the headline, the relevance is set to 1. When the item is an alert (urgency == 1), relevance should be gauged by firstMentionSentence instead.
* sentimentClass(int8) - indicates the predominant sentiment class for this news item with respect to the asset. The indicated class is the one with the highest probability.
* sentimentNegative(float32) - probability that the sentiment of the news item was negative for the asset
* sentimentNeutral(float32) - probability that the sentiment of the news item was neutral for the asset
* sentimentPositive(float32) - probability that the sentiment of the news item was positive for the asset
* sentimentWordCount(int32) - the number of lexical tokens in the sections of the item text that are deemed relevant to the asset. This can be used in conjunction with wordCount to determine the proportion of the news item discussing the asset.
* noveltyCount12H(int16) - The 12 hour novelty of the content within a news item on a particular asset. It is calculated by comparing it with the asset-specific text over a cache of previous news items that contain the asset.
* noveltyCount24H(int16) - same as above, but for 24 hours
* noveltyCount3D(int16) - same as above, but for 3 days
* noveltyCount5D(int16) - same as above, but for 5 days
* noveltyCount7D(int16) - same as above, but for 7 days
* volumeCounts12H(int16) - the 12 hour volume of news for each asset. A cache of previous news items is maintained and the number of news items that mention the asset within each of five historical periods is calculated.
* volumeCounts24H(int16) - same as above, but for 24 hours
* volumeCounts3D(int16) - same as above, but for 3 days
* volumeCounts5D(int16) - same as above, but for 5 days
* volumeCounts7D(int16) - same as above, but for 7 days

In [ ]:
data = []
for asset in np.random.choice(market_train['assetName'].unique(), 10):
    asset_df = market_train[(market_train['assetName'] == asset)]

    data.append(go.Scatter(
        x = asset_df['time'].dt.strftime(date_format='%Y-%m-%d').values,
        y = asset_df['close'].values,
        name = asset
    ))
layout = go.Layout(dict(title = "Closing prices of 10 random assets",
                  xaxis = dict(title = 'Month'),
                  yaxis = dict(title = 'Price (USD)'),
                  ),legend=dict(
                orientation="h"))
py.iplot(dict(data=data, layout=layout), filename='basic-line')

In [ ]:
data = []
for i in [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]:
    price_df = market_train.groupby('time')['close'].quantile(i).reset_index()

    data.append(go.Scatter(
        x = price_df['time'].dt.strftime(date_format='%Y-%m-%d').values,
        y = price_df['close'].values,
        name = f'{i} quantile'
    ))
layout = go.Layout(dict(title = "Trends of closing prices by quantiles",
                  xaxis = dict(title = 'Month'),
                  yaxis = dict(title = 'Price (USD)'),
                  ),legend=dict(
                orientation="h"))
py.iplot(dict(data=data, layout=layout), filename='basic-line')

In [ ]:
data = []
for i in [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]:
    price_df = market_train.groupby('time')['returnsOpenNextMktres10'].quantile(i).reset_index()

    data.append(go.Scatter(
        x = price_df['time'].dt.strftime(date_format='%Y-%m-%d').values,
        y = price_df['returnsOpenNextMktres10'].values,
        name = f'{i} quantile'
    ))
layout = go.Layout(dict(title = "Trends of returnsOpenNextMktres10 by quantiles",
                  xaxis = dict(title = 'Month'),
                  yaxis = dict(title = 'Price (USD)'),
                  ),legend=dict(
                orientation="h"),)
py.iplot(dict(data=data, layout=layout), filename='basic-line')

In [ ]:
text = ' '.join(news_train['headline'].str.lower().values[-1000000:])
stop = set(stopwords.words('english'))
wordcloud = WordCloud(max_font_size=None, stopwords=stop, background_color='white',
                      width=1200, height=1000).generate(text)
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud)
plt.title('Top words in headline')
plt.axis("off")
plt.show()

In [ ]:
(news_train['urgency'].value_counts() / 1000000).plot('bar');
plt.xticks(rotation=30);
plt.title('Urgency counts (mln)');

In [ ]:
news_train['provider'].value_counts().head(10)

In [ ]:
for i, j in zip([-1, 0, 1], ['negative', 'neutral', 'positive']):
    df_sentiment = news_train.loc[news_train['sentimentClass'] == i, 'assetName']
    print(f'Top mentioned companies for {j} sentiment are:')
    print(df_sentiment.value_counts().head(5))
    print('')

**Preprocess Data**

In [ ]:
market_train = market_train.loc[market_train['time'] >= '2010-01-01 22:00:00+0000']
news_train = news_train.loc[news_train['time'] >= '2010-01-01 22:00:00+0000']

In [ ]:
def preprocess_news(news_train):
    drop_list = [
        'audiences', 'subjects', 'assetName',
        'headline', 'firstCreated', 'sourceTimestamp',
    ]
    news_train.drop(drop_list, axis=1, inplace=True)
    
    # Factorize categorical columns
    for col in ['headlineTag', 'provider', 'sourceId']:
        news_train[col], uniques = pd.factorize(news_train[col])
        del uniques
    
    # Remove {} and '' from assetCodes column
    news_train['assetCodes'] = news_train['assetCodes'].apply(lambda x: x[1:-1].replace("'", ""))
    return news_train

news_train = preprocess_news(news_train)

**Split Asset Codes in News Data**

In [ ]:
def unstack_asset_codes(news_train):
    codes = []
    indexes = []
    for i, values in news_train['assetCodes'].iteritems():
        explode = values.split(", ")
        codes.extend(explode)
        repeat_index = [int(i)]*len(explode)
        indexes.extend(repeat_index)
    index_df = pd.DataFrame({'news_index': indexes, 'assetCode': codes})
    del codes, indexes
    gc.collect()
    return index_df

index_df = unstack_asset_codes(news_train)
index_df.head()

In [ ]:
def merge_news_on_index(news_train, index_df):
    news_train['news_index'] = news_train.index.copy()

    # Merge news on unstacked assets
    news_unstack = index_df.merge(news_train, how='left', on='news_index')
    news_unstack.drop(['news_index', 'assetCodes', 'sourceId'], axis=1, inplace=True)
    return news_unstack

news_unstack = merge_news_on_index(news_train, index_df)
del news_train, index_df
gc.collect()
news_unstack.head(3)

**Group Data**

In [ ]:
gc.collect()
def group_news(news_frame):
    news_frame['date'] = news_frame.time.dt.date  # Add date column
    
    aggregations = ['mean']
    gp = news_frame.groupby(['assetCode', 'date']).agg(aggregations)
    gp.columns = pd.Index(["{}_{}".format(e[0], e[1]) for e in gp.columns.tolist()])
    gp.reset_index(inplace=True)
    # Set datatype to float32
    float_cols = {c: 'float32' for c in gp.columns if c not in ['assetCode', 'date']}
    return gp.astype(float_cols)

news_agg = group_news(news_unstack)
del news_unstack; gc.collect()
news_agg.head(3)

**Merge News with Market Data**

In [ ]:
market_train['date'] = market_train.time.dt.date
df = market_train.merge(news_agg, how='left', on=['assetCode', 'date'])
del market_train, news_agg
gc.collect()
df.head(3)

**Data Cleanup**

In [ ]:
def missing_value_impute(data):
    grouped_data = data.groupby('assetCode')
    for i in data.columns:
        if data[i].dtype == "object":
            data[i] = data[i].fillna("other")
        elif (data[i].dtype == "float32" or data[i].dtype == "float64"):
            data[i] = data[i].fillna(grouped_data[i].mean())
            data[i] = data[i].fillna(0)
        else:
            pass
    return data

In [ ]:
gc.collect()
missing_value_impute(df)

In [ ]:
df.isna().sum()

In [1]:
gc.collect()

NameError: name 'gc' is not defined

**Data split**

In [ ]:
date = df.date
num_target = df.returnsOpenNextMktres10.astype('float32')
bin_target = (df.returnsOpenNextMktres10 >= 0).astype('int8')
# universe = df.universe.astype('int8')
# Drop columns that are not features
universe_data = df['universe']
time_data = df['time']
df.drop([
    'returnsOpenNextMktres10', 
    'date', 
    'universe', 
    'assetCode', 
    'assetName', 
    'time'], 
        axis=1, inplace=True)
df = df.astype('float32')  # Set all remaining columns to float32 datatype
gc.collect()

In [ ]:
x_train, x_test, y_train, y_test, u_train, u_test, t_train, t_test = \
    train_test_split(df, bin_target, universe_data, time_data, 
                     test_size=0.10, shuffle=False, random_state=99)

**Baseline Model - ExtraTreesClassifier**

In [ ]:
def plot_model_performance(y, y_hat):
    print("R2 score: ", r2_score(y, y_hat))
    print('Mean Absolute Error:', mean_absolute_error(y, y_hat))
    mse = mean_squared_error(y, y_hat)
    print('Mean Squared Error:', mse)  
    print('Root Mean Squared Error:', np.sqrt(mse)) 

In [ ]:
def do_baseline_modeling(x_train, x_test, y_train, y_test):
    model = ExtraTreesClassifier(bootstrap=True, oob_score=True,
                                    n_estimators=10, class_weight="balanced_subsample")
    prev_oob_score = 0.0
    for i in range(1, 10):
        gc.collect()
        model.set_params(n_estimators=i)    
        model.fit(x_train, y_train)
        print(model.oob_score_)
        if i > 1 and (model.oob_score_ - prev_oob_score < 0.05):
            gc.collect()
            break
        prev_oob_score = model.oob_score_
        gc.collect()
    y_hat = model.predict(x_test)
    plot_model_performance(y_test, y_hat)
    return model

In [ ]:
baseline_model = do_baseline_modeling(x_train, x_test, y_train, y_test)
imp_features = baseline_model.feature_importances_
print(imp_features)

**Hyper parameter Tuning**

In [ ]:
def sigma_score(preds, valid_data):
    df_time = valid_data.params['extra_time']
    labels = valid_data.get_label()
    x_t = preds * labels
    x_t_sum = x_t.groupby(df_time).sum()
    score = x_t_sum.mean() / x_t_sum.std()
    return 'sigma_score', score, True

In [ ]:
def f(x):
    gc.collect()
    print(x)
    params = {
        'task': 'train',
        'boosting_type': 'dart',
        'learning_rate': x[0],
        'num_leaves': x[1],
        'min_data_in_leaf': x[2],
        'num_iteration': x[3],
        'max_bin': x[4],
        'verbose': 1,
        'objective': 'binary'
    }
    d_train = lgb.Dataset(x_train, label=y_train)
    d_test = lgb.Dataset(x_test, label=y_test)
    gbm = lgb.train(params,
            d_train,
            num_boost_round=100,
            valid_sets=d_test,
            early_stopping_rounds=5)
    
    print('score: ', mean_squared_error(
        gbm.predict(x_test, num_iteration=gbm.best_iteration), y_test))
    gc.collect()
    return mean_squared_error(
        gbm.predict(x_test, num_iteration=gbm.best_iteration), y_test)

# optimize params in these ranges
spaces = [
    (0.01, 0.3), #learning_rate
    (3000, 3500), #num_leaves
    (260, 300), #min_data_in_leaf
    (800, 1000), #num_iteration
    (200, 220) #max_bin
]

# run optimization
res = gp_minimize(
    f, spaces,
    acq_func="EI",
    n_calls=10) # increase n_calls for more performance

# print tuned params
print(res.x)

# plot tuning process
plot_convergence(res)

In [ ]:
params = {
    'learning_rate': res.x[0],
    'num_leaves': res.x[1],
    'n_estimators': 200,
    'min_child_samples': 20,
    'colsample_bytree': 0.8,
    'subsample': 1.0,
    'reg_alpha': 0.8,
    'reg_lambda': 0.4,
    'task': 'train',
    'boosting_type': 'dart',
    'min_data_in_leaf': res.x[2],
    'num_iteration': res.x[3],
    'max_bin': res.x[4],
    'verbose': 1,
    'objective': 'binary'
}
params

**Train LGBM Model**

In [ ]:
# params = {
#     'learning_rate': 0.14360424246380565,
#     'num_leaves': 170,
#     'n_estimators': 200,
#     'min_child_samples': 1500,
#     'colsample_bytree': 0.8,
#     'subsample': 1.0,
#     'reg_alpha': 0.8,
#     'reg_lambda': 0.4,
#     'task': 'train',
#     'boosting_type': 'dart',
#     'min_data_in_leaf': 500,
#     'num_iteration': 50,
#     'max_bin': 50000,
#     'verbose': 1,
#     'objective': 'binary'
# }

In [ ]:
gc.collect()
d_train = lgb.Dataset(x_train, label=y_train)
d_test = lgb.Dataset(x_test, label=y_test)
clf = lgb.train(params,
        d_train,
        num_boost_round=1000,
        valid_sets=d_test,
        early_stopping_rounds=5)

In [ ]:
gc.collect()

**Evaluate LGBM Model**

In [ ]:
def evaluate_lgbm_model(x_train, x_test, y_train, y_test, params, model):
    gc.collect()
    d_train = lgb.Dataset(x_train, label=y_train)
    d_test = lgb.Dataset(x_test, label=y_test)
    watchlist = [d_test]
    model = lgb.train(params, d_train, watchlist, verbose_eval=1)
    plot_model_performance(y_test, model.predict(x_test))

In [2]:
evaluate_lgbm_model(x_train, x_test, y_train, y_test, params, clf)

NameError: name 'evaluate_lgbm_model' is not defined

**Feature Importance**

In [ ]:
feat_importance = pd.DataFrame()
feat_importance["feature"] = df.columns
feat_importance["gain"] = clf.feature_importance(importance_type='gain')
feat_importance.sort_values(by='gain', ascending=False, inplace=True)
plt.figure(figsize=(8,10))
ax = sns.barplot(y="feature", x="gain", data=feat_importance)

**Write Submission**

In [ ]:
days = env.get_prediction_days()

In [ ]:
def write_submission(model, env, days):
    for (market_obs_df, news_obs_df, predictions_template_df) in days:
        news_obs_df = preprocess_news(news_obs_df)
        # Unstack news
        index_df = unstack_asset_codes(news_obs_df)
        news_unstack = merge_news_on_index(news_obs_df, index_df)
        # Group and and get aggregations (mean)
        news_obs_agg = group_news(news_unstack)

        # Join market and news frames
        market_obs_df['date'] = market_obs_df.time.dt.date
        obs_df = market_obs_df.merge(news_obs_agg, how='left', on=['assetCode', 'date'])
        del market_obs_df, news_obs_agg, news_obs_df, news_unstack, index_df
        gc.collect()
        obs_df = obs_df[obs_df.assetCode.isin(predictions_template_df.assetCode)]
        dropped_features =  ['universe', 'date', 
                                             'assetCode', 'assetName', 'time']
        feats = [c for c in obs_df.columns if c not in dropped_features]

        preds = model.predict(obs_df[feats])[:] * 2 - 1
        sub = pd.DataFrame({'assetCode': obs_df['assetCode'], 'confidence': preds})
        predictions_template_df = predictions_template_df.merge(sub, how='left').drop(
            'confidenceValue', axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
        
        env.predict(predictions_template_df)
        del obs_df, predictions_template_df, preds, sub
        gc.collect()
    env.write_submission_file()

if 'days' not in globals():
    days = env.get_prediction_days()   
write_submission(clf, env, days)